In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9 

     |████████████████████████████████| 204.2 MB 29 kB/s 
     |████████████████████████████████| 198 kB 51.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612246 sha256=e27b57963d9b0ee11e9155b8946171473057be30136f8096d1fd4b9e6bd5b4a8
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.stat import Correlation
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .master("local[4]") \
    .config("spark.driver.maxResultSize", "8g") \
    .getOrCreate()

In [3]:
from google.colab import drive               
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
df = spark.read.parquet('../../data/cleanedDataset_parquet')

In [3]:
df.show(10)

+--------------------+----------------+-----------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+---------------------+--------------------+---------------------+------------------+
|            id_track|popularity_track|duration_ms|              genres|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|sum_artist_followers|sum_artist_popularity|avg_artist_followers|avg_artist_popularity|               age|
+--------------------+----------------+-----------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+---------------------+--------------------+---------------------+------------------+
|00AeAaSNbe92PRrst...|               3|     156067|[classic czech p

In [4]:
df.printSchema()

root
 |-- id_track: string (nullable = true)
 |-- popularity_track: integer (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- release_date: date (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: integer (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- acousticness: float (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- liveness: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- tempo: float (nullable = true)
 |-- time_signature: integer (nullable = true)
 |-- sum_artist_followers: long (nullable = true)
 |-- sum_artist_popularity: long (nullable = true)
 |-- avg_artist_followers: double (nullable = true)
 |-- avg_artist_popularity: double (nullable = true)
 |-- age: double (nullable = true)



In [5]:
from pyspark.ml.feature import QuantileDiscretizer 

qds = QuantileDiscretizer(relativeError=0.0001, handleInvalid="error", numBuckets=10, inputCol="popularity_track", outputCol="label")

df = qds.setHandleInvalid("keep").fit(df).transform(df)

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
assembler = VectorAssembler( 
inputCols=[
 'duration_ms',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'avg_artist_followers',
 'avg_artist_popularity',
 'sum_artist_followers',
 'sum_artist_popularity'], 
outputCol="features")
output=assembler.transform(df)

In [8]:
from pyspark.ml.feature import MinMaxScaler
scaler = MinMaxScaler(inputCol = "features",outputCol= "scaledFeatures")
scalerData = scaler.fit(output)
out = scalerData.transform(output)

In [10]:
final_data = out.select("id_track", "scaledFeatures", "label")

In [11]:
train, test = final_data.randomSplit([0.7, 0.3], seed = 10)

In [12]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol="scaledFeatures",labelCol="label",maxDepth = 10)
rfModel = rf.fit(train)
predictions = rfModel.transform(test)

In [13]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.30365019315153696

# RF with filtered dataset

In [14]:
df = spark.read.parquet('../../data/cleanedDatasetFiltered_parquet')

qds = QuantileDiscretizer(relativeError=0.0001, handleInvalid="error", numBuckets=10, inputCol="popularity_track", outputCol="label")

df = qds.setHandleInvalid("keep").fit(df).transform(df)

assembler = VectorAssembler( 
inputCols=[
 'duration_ms',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'avg_artist_followers',
 'avg_artist_popularity',
 'sum_artist_followers',
 'sum_artist_popularity'], 
outputCol="features")
output=assembler.transform(df)

scaler = MinMaxScaler(inputCol = "features",outputCol= "scaledFeatures")
scalerData = scaler.fit(output)
out = scalerData.transform(output)

final_data = out.select("id_track", "scaledFeatures", "label")

train, test = final_data.randomSplit([0.7, 0.3], seed = 10)

rf = RandomForestClassifier(featuresCol="scaledFeatures",labelCol="label",maxDepth = 10)
rfModel = rf.fit(train)
predictions = rfModel.transform(test)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.2652434322935621